<a href="https://colab.research.google.com/github/tomheston/fragility-metrics/blob/main/notebooks/binary_2x2_fixed_margin.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [13]:
# @title
# Fragility Metrics Toolkit: Matched / Fixed-Margin Binary Outcomes (2×2)
# 19-NOV-2025
#
# Layout (generic 2×2 fixed-margin / matched design):
#      | Outcome A | Outcome B |
# ArmA |     a     |    b      |
# ArmB |     c     |    d      |
#
# Input:
#   a, b, c, d  (non-negative integers; margins fixed by design)
#
# Output (p–fr–nb):
#   p   : McNemar χ² baseline p-value (two-sided, uncorrected)
#   fr  : PFI (Percent Fragility Index, decimal scale) via McNemar path
#   nb  : RQ  (Risk Quotient, Neutrality Boundary Framework)
#
# Notes
# - This calculator is for matched / fixed-margin 2×2 designs.
# - Baseline significance test: McNemar χ² (two-sided, df = 1, no correction).
# - PFI is computed along the fixed-margin path:
#       (a, b, c, d) → (a + x, b - x, c - x, d + x)
#   with continuous x. Along this path, McNemar p-value is strictly monotonic:
#   p(x) decreases as x increases.
#   Normalization: 4 × |x| / N → [0,1] scale, orthogonal to RQ.
#
# IF YOU USE THIS CALCULATOR PLEASE CITE:
# Heston, T. F. (2025). Fragility Metrics Toolkit [Software]. Zenodo.
# https://doi.org/10.5281/zenodo.17254763
#
# © Thomas F. Heston 2025. CC-BY 4.0

# ----- SciPy availability guard -----
try:
    import scipy
except ImportError:
    try:
        import subprocess
        import sys
        subprocess.check_call([sys.executable, "-m", "pip", "install", "-q", "scipy"])
        import scipy
    except Exception:
        print("Please install scipy: pip install scipy")
        raise

import numpy as np
from scipy.stats import chi2

ALPHA = 0.05


# ---------- Core utilities ----------

def n_total(a, b, c, d):
    """Total sample size."""
    return a + b + c + d


def mcnemar_p(a, b, c, d):
    """
    McNemar χ² test (two-sided, uncorrected).
    χ² = (b - c)² / (b + c), df = 1, no continuity correction.
    """
    b_c = b + c
    if b_c == 0:
        return 1.0
    chi2_stat = (b - c) ** 2 / b_c
    p = chi2.sf(chi2_stat, df=1)
    return float(p)


def p_along_x_mcnemar(a, b, c, d, x):
    """
    McNemar p-value along the fixed-margin path:
    (a, b, c, d) → (a + x, b - x, c - x, d + x)
    """
    aa, bb, cc, dd = a + x, b - x, c - x, d + x
    if min(aa, bb, cc, dd) < 0:
        return np.nan
    return mcnemar_p(aa, bb, cc, dd)


# ---------- PFI + RQ (fixed-margin, McNemar path) ----------

def compute_pfi_and_rq(a, b, c, d, alpha=ALPHA, tol=1e-12, max_iter=300):
    a, b, c, d = map(float, (a, b, c, d))
    n = a + b + c + d
    if n <= 0 or min(a, b, c, d) < 0:
        return {"PFI": None, "RQ": None, "boundary_limited": False}

    # Robustness (nb) – distance from marginal homogeneity (McNemar null)
    # Used only in matched-pair / fixed-margin module for internal consistency
    discordants = b + c
    if discordants == 0:
        nb = 0.0
    else:
        nb = abs(b - c) / discordants

    # Feasible shift range
    x_min = -min(a, d)
    x_max = min(b, c)

    # Baseline p-value
    p0 = mcnemar_p(a, b, c, d)
    if not np.isfinite(p0):
        return {"PFI": None, "MHQ": nb, "boundary_limited": False}

    sig0 = (p0 <= alpha)

    def f(x):
        return p_along_x_mcnemar(a, b, c, d, x) - alpha

    # Monotonicity: p(x) decreases as x increases
    if sig0:
        lo, hi = x_min, 0.0
        boundary_x = x_min
    else:
        lo, hi = 0.0, x_max
        boundary_x = x_max

    if abs(lo - hi) < tol:
        return {"PFI": None, "RQ": rq, "boundary_limited": False}

    flo, fhi = f(lo), f(hi)

    # Check if flip is impossible even at boundary
    if flo * fhi > 0:
        p_boundary = p_along_x_mcnemar(a, b, c, d, boundary_x)
        if (sig0 and p_boundary <= alpha) or (not sig0 and p_boundary > alpha):
            # Maximum robustness case
            pfi = 4.0 * abs(boundary_x) / n
            return {"PFI": pfi, "MHQ": nb, "boundary_limited": True}
        return {"PFI": None, "MHQ": nb, "boundary_limited": False}

    if not (np.isfinite(flo) and np.isfinite(fhi)):
        return {"PFI": None, "MHQ": nb, "boundary_limited": False}

    # Bisection (normal case)
    for _ in range(max_iter):
        mid = 0.5 * (lo + hi)
        fmid = f(mid)
        if not np.isfinite(fmid):
            mid = np.nextafter(mid, hi)
            fmid = f(mid)
            if not np.isfinite(fmid):
                break
        if abs(hi - lo) < tol or fmid == 0:
            x_star = mid
            break
        if flo * fmid <= 0:
            hi, fhi = mid, fmid
        else:
            lo, flo = mid, fmid
    else:
        x_star = 0.5 * (lo + hi)

    pfi = 4.0 * abs(x_star) / n
    return {"PFI": pfi, "RQ": rq, "boundary_limited": False}

# ---------- High-level calculator ----------

def calculate_binary_2x2_fixed_margin(a, b, c, d, alpha=ALPHA):
    p_val = mcnemar_p(a, b, c, d)
    result = compute_pfi_and_rq(a, b, c, d, alpha=alpha)
    return {
        "p": p_val,
        "fr": {"PFI": result["PFI"], "boundary_limited": result["boundary_limited"]},
        "nb": {"MHQ": result["MHQ"]}
    }


# ---------- CLI ----------

def main():
    print("Enter 2×2 fixed-margin table cells as integers.")
    a = int(input("a (Arm A outcome A): ").strip())
    b = int(input("b (Arm A outcome B): ").strip())
    c = int(input("c (Arm B outcome A): ").strip())
    d = int(input("d (Arm B outcome B): ").strip())

    res = calculate_binary_2x2_fixed_margin(a, b, c, d)
    N = n_total(a, b, c, d)

    print("\n================ p–fr–nb =================")
    print(f"N = {n_total(a, b, c, d)}")
    print(f"p = {res['p']:.5f} (McNemar χ² two-sided)")
    #print("fr:")
    pfi_val = res["fr"]["PFI"]
    if pfi_val is not None:
        note = " (boundary-limited: no admissible shift flips significance)" if res["fr"]["boundary_limited"] else " (PFI)"
        print(f"fr = {pfi_val:.6f}{note}")
    else:
        print("fr = None")
    #print("nb:")
    print(f"nb = {res['nb']['MHQ']:.6f} (distance from marginal homogeneity per MHQ)")
    print("==========================================")

if __name__ == "__main__":
    main()


Enter 2×2 fixed-margin table cells as integers.
a (Arm A outcome A): 310
b (Arm A outcome B): 2273
c (Arm B outcome A): 92
d (Arm B outcome B): 2730

================ p–fr–nb =================
N = 5405
p = 0.00000 (McNemar χ² two-sided)
fr = 0.229417 (boundary-limited: no admissible shift flips significance)
nb = 0.922199 (MHQ)
